In [1]:
import torch
import torchvision.models as models

Save & Load models

Tham số đã học được lưu trữ trong một từ điển trạng thái nội bộ - state_dict. Những tham số này được lưu thông qua torch.save

In [2]:
model = models.vgg16(weights='IMAGENET1K_V1')
torch.save(model.state_dict(), 'model_weights.pth')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/qk/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100.0%


model = models.vgg16(weights='IMAGENET1K_V1')
- models.vgg16: lấy mô hình VGG16, là kiến trúc CNN dùng trong bài toán nhận dạng hình ảnh
- weights="IMAGENET1K_V1": ta đang tải trọng số đc huấn luyện sẵn trên tập ImageNet (1K lớp phân loại) 

=> Đây là cách TRANSFER LEARNING hoạt động - không cần huấn luyện lại từ đầu
    - Transfer learning: kỹ thuật tái sử dụng mô hình đã được huấn luyện trước đó trên một bài toán tương tự, sau đó fine-tune hoặc tái sử dụng một phần nào đó của mô hình cho bài toán mới.

torch.save(model.state_dict(), 'model_weights.pth')
- model.state_dict(): trích xuất tất cả weight và bias của mô hình dưới dạng OrderedDict
- torch.save(..., 'model_weights.pth'): lưu lại trọng số  vào file chỉ định trên ổ đĩa.
    
    => Không lưu toàn bộ mô hình, chỉ lưu trọng số - thường dùng khi tái tạo lại kiến trúc mô hình và load lại trọng số

Để load, ta cần một phiên bản của cùng một mô hình, sau đó load bằng method load_state_dict()

Pickling/Unpickling 
- Pickling: quá trình chuyển đổi python obj sang byte stream để lưu vào file hoặc truyền qua mạng.
    - PyTorch dùng pickle để lưu mô hình hoặc state_dict
- Unpickling: quá trình đọc lại(giải mã) file nhị phân đó và khôi phục lại Python obj ban đầu 

In [ ]:
# ví dụ minh họa
# torch.save(obj, 'file.pth')     # Pickling
# obj = torch.load('file.pth')    # Unpickling

weights_only=True để giới hạn các hàm được thực hiện trong quá trình unpickling - chỉ thực hiện những tác vụ cần thiết để tải trọng số. 
- Không có weights_only=True:
    - Unpickle toàn bộ nội dung file:
        - weight(state_dict)
        - Các hàm hoặc class đã được định nghĩa khi lưu.
        - có thể có các obj tùy chỉnh mà ta không cần
    => không an toàn nêus file chưa mã độc. vì unpickling có thể chạy code ngầm
- weights_only=True:
    - giải nén các tensor chứa trọng số mô hình (weights, bias)
    - Bỏ qua mọi đối tượng phức tạp như:
        - class tự định nghĩa
        - hàm tùy chỉnh
        - mô hình full (với các hàm forward)
    - Không tạo lại toàn bộ obj đã được pickle, chỉ đọc phần dữ liệu cần thiết để tạo state_dict



Đây là phương pháp hay nhất khi tải trọng số:
- Không xử lý phần không cần thiết
- Nhanh hơn, an toàn
- Chính xác hơn: phù hợp với các use case như transfer learning khi ta chỉ cần load trọng số vào models mà ta định nghĩa

In [4]:
model = models.vgg16() # we do not specify ``weights``, i.e. create untrained model
model.load_state_dict(torch.load('model_weights.pth', weights_only=True))
model.eval()
# => Ta chỉ tạo backbone để load weight, bias vào 

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

Output là kiến trúc đầy đủ cảu mô hình VGG16 gồm 3 phần chính
- features (BACKBONE) => phần trích xuất đặc trưng
    - Gồm Conv2d, ReLU, MaxPool2d lặp lại nhiều lần
    - Mục tiêu: trích xuất các đặc trưng từ ảnh đầu vào
    - Input thường là ảnh RGB (3 kênh), output là tensor nhiều chiều có nhiều kênh
- avgpool
    - Dùng AdaptiveAvgPool2d để điều chỉnh kích thước đầu ra cố định là (7, 7)
    - Chuẩn bị cho việc đưa vào Linear phía sau
- classifier (HEAD) => phần phân loại
    - Gồm các lớp Linear, ReLU, Dropout
    - Biến vector đặc trưng → phân loại 1000 lớp (ImageNet): 25088 (512×7×7) → 4096 → 4096 → 1000

Dropout là kỹ thuật regularization trong mạng neural - dùng để giảm hiện tượng overfitting trong quá trình huấn luyện

Cách hoạt động:
- Trong mỗi lần huấn luyện, Dropout sẽ ngẫu nhiên "tắt" một neural trong mạng
- Tức là: một số node trong layer sẽ không tham gia tính toán hoặc truyền grad ở lượt đó
=> điều này giúp mô hình:
- không quá phụ thuộc vào một nhóm neural cụ thể
- học được những biểu diễn đa dạng và tổng quát hơn

Khi nào sẽ hoạt động: 
- model.train() -> dropout hoạt động để regularize mô hình
- model.eval() -> dropout không hoạt động để mô hình hđ ổn định

Ghi chú: chắc chắn gọi phương thức model.eval() trước khi suy luận để đặt các lớp chuẩn hóa dropout và batch sang chế độ đánh giá. Nếu quên gọi:
- Mô hình sẽ cập nhật hoặc cộng dồn các giá trị (hành động như được train) 
- Điều này làm cho kết quả dự đoán bị nhiễu loạn, không ổn định, và sai lệch

SAVE & LOAD MODELS

Khi save weight, ta cần tạo lớp models trước vì lớp này định nghĩa cái backborn của mạng. Ta có thể truyền model nếu muốn lưu cấu trúc cùng với mô hình (ngược lại truyền model.state_dict())

In [ ]:
torch.save(model, 'model.pth')

In [ ]:
model = torch.load('model.pth', weights_only=False),
# weights_only=False vì đây là hành động liên quan đến việc tải mô hình.

Nên dùng weights_only = False khi:
- Lưu toàn bộ mô hình bằng torch.save(model) (pickle full object)
- Muốn load nguyên mô hình, không cần định nghĩa lại kiến trúc

Ghi chú:

Khi dùng torch.save(model, 'model.pth'), PyTorch dùng pickle để lưu toàn bộ model obj: weight, bias, class, func
Khi load lại model = torch.load('full_model.pth'), Pytorch dùng pickle để tái tạo lại đúng class mô hình cũ. 
- nếu định nghĩa lớp không còn tồn tại hoặc dùng máy khác không có code gốc => lỗi

Nên khi dùng torch.save(model) thì phải đúng class mô hình gốc khi load. 

Nếu chỉ lưu/ truyền trọng số (state_dict) thì không cần class mô hình gốc => định nghĩa đúng kiến trúc tương đồng là xong.